# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 13** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez </br>
**Team**: Par de Dos </br>
**Members**: Diego Orozco and Aarón Ortega


### FindSpark

In [1]:
import findspark
findspark.init()

### Connection with SparkCluster

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab13_ParDeDos") \
    .master("spark://368ad5a83fd7:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/05 03:05:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Schema and DataFrame Creation

In [4]:
from team_ParDeDos.spark_utils import SparkUtils as SpU

schema = SpU.generate_schema([
  ("Alcohol", "double"),
  ("Malic_Acid", "double"),
  ("Ash", "double"),
  ("Ash_Alcanity", "float"),
  ("Magnesium", "integer"),
  ("Total_Phenols", "double"),
  ("Flavanoids", "double"),
  ("Nonflavanoid_Phenols", "double"),
  ("Proanthocyanins", "double"),
  ("Color_Intensity", "double"),
  ("Hue", "double"),
  ("0D280", "double"),
  ("Proline", "integer")
])

wine_df = spark \
          .read \
          .schema(schema) \
          .option("header", "true") \
          .option("mode", "dropMalformed") \
          .csv("/home/jovyan/notebooks/data/wine-clustering.csv")

### Assemble the features into a single vector column

In [7]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["Alcohol", "Malic_Acid", "Ash", "Ash_Alcanity", "Magnesium", "Total_Phenols", "Flavanoids", "Nonflavanoid_Phenols", "Proanthocyanins", "Color_Intensity", "Hue", "0D280", "Proline"], outputCol="features")

assembled_df = assembler.transform(wine_df)

### Initialize KMeans

In [8]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(3).setSeed(19)

### Training

In [9]:
model = kmeans.fit(assembled_df)

25/05/05 03:06:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/05 03:07:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/05 03:07:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


### Predictions

In [10]:
predictions = model.transform(assembled_df)

### Evaluation Model

In [11]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette Score: {silhouette}")

# Show results
print("Cluster Centers: ")
for center in model.clusterCenters():
  print(center)

Silhouette Score: 0.7274114110259972
Cluster Centers: 
[1.25985294e+01 2.45343137e+00 2.32186275e+00 2.06460784e+01
 9.36960784e+01 2.05362745e+00 1.64754902e+00 3.95980392e-01
 1.42509804e+00 4.67333332e+00 9.17843137e-01 2.39480392e+00
 5.21558824e+02]
[1.33691837e+01 2.40000000e+00 2.39265306e+00 1.85142857e+01
 1.09081633e+02 2.44163265e+00 2.21367347e+00 3.25510204e-01
 1.70673469e+00 5.18836735e+00 9.59714286e-01 2.84795918e+00
 9.06346939e+02]
[1.38507407e+01 1.77851852e+00 2.48777778e+00 1.69259260e+01
 1.05629630e+02 2.94148148e+00 3.13666667e+00 2.98888889e-01
 2.00703704e+00 6.27518519e+00 1.10296296e+00 3.00222222e+00
 1.30877778e+03]


In [12]:
sc.stop()